In [ ]:
from google.colab import drive
drive.mount('/content/drive')
home_directory = '/content/drive/MyDrive/Aps360 Project/'

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install anvil-uplink
!pip install datasets
!pip install git+https://github.com/huggingface/transformers
!pip install rouge_score

     |████████████████████████████████| 2.6 MB 15.1 MB/s 
     |████████████████████████████████| 895 kB 55.6 MB/s 
     |████████████████████████████████| 636 kB 72.0 MB/s 
     |████████████████████████████████| 3.3 MB 51.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 62 kB 981 kB/s 
     |████████████████████████████████| 51 kB 209 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=cd117ebb4ec8dbce4709567f328a1b40dc7703cfe75df93e66a7a5ca49eec389
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


     |████████████████████████████████| 264 kB 15.9 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 118 kB 49.6 MB/s 
     |████████████████████████████████| 243 kB 70.6 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-s3_g5z99
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-s3_g5z99
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.10.0.dev0-py3-none-any.whl size=2633491 sha256=a346d315edb31ff1c184fec32b54848792dc53a9cebc22cabca487820665b0a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-zud2o0dm/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully 

In [ ]:
import numpy as np
import pandas as pd 
import csv

import random
import time
import datetime
import gc

import anvil.server
from anvil.google.drive import app_files


import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, pipeline


In [ ]:
#connect to anvil
anvil.server.connect("63WH32SPMQZOLFWQPLXJ2J66-M6YKH5VSTGUY2N2N")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [ ]:
RANDOM_SEED = 73
MAX_LEN = 600

##Setup Tokenizer

In [ ]:

#Load the GPT2 tokenizer that will be used by PoemDataset to encode the poems. Add the 
#BOS,EOS and PAD tokens to the tokenized dictionary so that when we put these 
#Tokens around our poems to separate them, the tokenizer will know what to do with them.
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'sep_token':'<SEP>', 'pad_token': '<PAD>'}
tokenizer.add_special_tokens(special_tokens_dict)


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

4

In [ ]:
torch.cuda.manual_seed_all(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device('cuda')

gc.collect()
torch.cuda.empty_cache() 

#If you are only evaluating the model and don't want to train it again,
#set training_desired to false. Otherwise set it to true and choose the 
#epoch you left off at last time to start Training from (0 if you haven't).
previous_state_dict_location = "model_amazon_review_poem_FINAL_part2"

#Setup the pretrained GPT2 model
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions=MAX_LEN).from_pretrained('gpt2', output_hidden_states=True)
poem_model = GPT2LMHeadModel.from_pretrained('gpt2', config=configuration)
poem_model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Embedding(50261, 768)

In [ ]:
poem_model.load_state_dict(torch.load("{}/{}".format(home_directory,previous_state_dict_location)))

poem_model.cuda()
poem_model = poem_model.to(device)

##Use the model to generate poems

In [ ]:
poem_model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [ ]:
def generate_poem(prompt):
  #first, write the prompt to a test file
  with open('test.csv', 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_ALL)
    writer.writerow(['summary','text'])
    writer.writerow(['unknown',prompt])

  #then create the generated title of the review
  !python "/content/drive/MyDrive/Aps360 Project/reviewSummarizer/run_summarization.py" \
  --do_predict \
  --test_file "/content/test.csv" \
  --model_name_or_path "t5-small" \
  --do_train \
  --train_file "/content/drive/MyDrive/Aps360 Project/Datasets/amazonReviews/train/train_100.csv" \
  --source_prefix "summarize: " \
  --output_dir "/content/drive/MyDrive/Aps360 Project/reviewSummarizer" \
  --overwrite_output_dir \
  --per_device_train_batch_size=20 \
  --per_device_eval_batch_size=20 \
  --predict_with_generate \
  --text_column text \
  --summary_column summary \
  --max_train_samples 1 \
  --resume_from_checkpoint "/content/drive/MyDrive/Aps360 Project/reviewSummarizer/checkpoint-5000-safe"

  #load the generated title:
  file1 = open('/content/drive/MyDrive/Aps360 Project/reviewSummarizer/generated_predictions.txt', 'r',encoding="utf-8")
  lines = file1.readlines()
  prompt_summary = lines[0]

  original_summary = prompt_summary           
  prompt_summary = prompt_summary + "<SEP>"

  encoded = torch.tensor(tokenizer.encode(prompt_summary)).unsqueeze(0)
  generated = encoded.to(device)
  sample_outputs = poem_model.generate(
                                  generated, 
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length=MAX_LEN,
                                  top_p=0.95, 
                                  num_return_sequences=1
                                  )
  return original_summary.upper() + '\n' + tokenizer.decode(sample_outputs[0], skip_special_tokens=True)[len(original_summary):]
  

In [ ]:
@anvil.server.callable
def poetify(review):
  result = generate_poem(review)
  return result

In [ ]:
anvil.server.wait_forever()

2021-08-07 05:50:24.827950: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
08/07/2021 05:50:26 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/07/2021 05:50:26 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
l

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: ignored